In [1]:
from pymongo import MongoClient
import pandas as pd

import src.googla as gl
import os
from dotenv import load_dotenv
import json
import requests
import src.filtering as fl

load_dotenv()

True

In [2]:
conn = MongoClient("localhost:27017")

#### Comprobamos que existe y aparece la base de datos "Cleaned_offices" que necesitamos utilizar, así como la colección correspondiente

In [3]:
conn.list_database_names()

['Cleaned_offices',
 'Employees',
 'Ironhack',
 'PullRequests',
 'admin',
 'companies',
 'config',
 'local']

In [4]:
db = conn.get_database("Cleaned_offices")

In [5]:
db.list_collection_names()

['places', 'can_offices', 'collection', 'cleaned_offices']

In [6]:
collection = db.get_collection("cleaned_offices")


In [7]:
collection.find_one()


{'_id': ObjectId('5fb0325b71eb61051a67a4e3'),
 'name': 'Geni',
 'category_code': 'web',
 'founded_year': 2006,
 'offices': {'description': 'Headquarters',
  'address1': '9229 W. Sunset Blvd.',
  'address2': '',
  'zip_code': '90069',
  'city': 'West Hollywood',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 34.090368,
  'longitude': -118.393064},
 'total_funding': 16500000,
 'city': 'West Hollywood',
 'latitude': 34.090368,
 'longitude': -118.393064,
 'location': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]}}

#### Comprobamos las keys que hay 

In [8]:
collection.find_one().keys()


dict_keys(['_id', 'name', 'category_code', 'founded_year', 'offices', 'total_funding', 'city', 'latitude', 'longitude', 'location'])

### Después de investigar al respecto  e informarme acerca de la situación tecnológica de algunas de las economías más fuertes del mundo, he decidido situar la oficina en Vancouver, Canadá. Esto se debe, por un lado, a que[ el sector tecnológico de la ciudad está en pleno auge](https://www.theglobeandmail.com/business/industry-news/property-report/article-vancouvers-emerging-tech-sector-poised-for-accelerated-growth/), llamando la atención de empresas como MIcrosoft Corp. Sony Pictures Imageworks Inc. o Amazon.  Además, por otra parte, se espera que [la fuerte economía en crecimiento de Vancouver en la situación pre-COVID se recupere casi en su totalidad](https://www.vancouvereconomic.com/economic-snapshot/), cuestión interesante teniendo en cuenta la crisis económica que seguirá a la sanitaria que se está viviendo a nivel mundial.

In [12]:
can_offices = collection.find({"offices.country_code": 'CAN', "city": "Vancouver"})

In [13]:
offices_list = list(can_offices)

In [14]:
offices_list[0]

{'_id': ObjectId('5fb0325b71eb61051a67a560'),
 'name': 'NowPublic',
 'category_code': 'web',
 'founded_year': 2005,
 'offices': {'description': '',
  'address1': '576 Seymour Street',
  'address2': 'Suite 204',
  'zip_code': '',
  'city': 'Vancouver',
  'state_code': None,
  'country_code': 'CAN',
  'latitude': 49.283376,
  'longitude': -123.103986},
 'total_funding': 12000000,
 'city': 'Vancouver',
 'latitude': 49.283376,
 'longitude': -123.103986,
 'location': {'type': 'Point', 'coordinates': [-123.103986, 49.283376]}}

In [15]:
#def get_google(latlong, radius,typ, apiKey, keyw=None):
  #  keyword= '&keyword='+keyw if keyw else ''
  #  url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latlong}&radius={radius}&type={typ}{keyword}&key={apiKey}'
 #   res = requests.get(url)
 #   return res.json()['results']

In [16]:
#def get_latlong(office):
  #  return str(office['location']['coordinates'][::-1]).replace(' ', '')[1:-1]

#### Utilizamos una función ("fill_nearest_places") para llamar a la API de Google Places y, a partir de la lista de oficinas filtradas en el paso anterior,  obtener de cada oficina los aeropuertos, los Starbucks, los clubs nocturnos y los colegios y realizamos un "len" de la lista para así saber cuántos de cada tiene cada oficina en el radio que le marcamos:

-  Aeropuertos (internacionales) en un radio de 2 km. Especificamos internacionales porque Vancouver tiene algunos aeropuertos para sea-planes.y así poder viajar entre islas cercanas.

- Colegios en un radio de 1 km.

- Clubs nocturnos en un radio de 1 km.

- Starbucks en un radio de 500 metros.


In [17]:
filled_offices_list = gl.fill_nearest_places(offices_list)

In [18]:
filled_offices_list[0]

{'_id': ObjectId('5fb0325b71eb61051a67a560'),
 'name': 'NowPublic',
 'category_code': 'web',
 'founded_year': 2005,
 'offices': {'description': '',
  'address1': '576 Seymour Street',
  'address2': 'Suite 204',
  'zip_code': '',
  'city': 'Vancouver',
  'state_code': None,
  'country_code': 'CAN',
  'latitude': 49.283376,
  'longitude': -123.103986},
 'total_funding': 12000000,
 'city': 'Vancouver',
 'latitude': 49.283376,
 'longitude': -123.103986,
 'location': {'type': 'Point', 'coordinates': [-123.103986, 49.283376]},
 'airports_2km': 5,
 'primary_schools_1km': 2,
 'night_clubs_1km': 20,
 'starbucks_500m': 5}

In [19]:
filled_offices_df = pd.DataFrame(filled_offices_list)

In [20]:
filled_offices_df.head(3)

,_id,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location,airports_2km,primary_schools_1km,night_clubs_1km,starbucks_500m
0,5fb0325b71eb61051a67a560,NowPublic,web,2005.0,"{'description': '', 'address1': '576 Seymour S...",12000000,Vancouver,49.283376,-123.103986,"{'type': 'Point', 'coordinates': [-123.103986,...",5,2,20,5
1,5fb0325b71eb61051a67a68c,TeamPages,web,2006.0,"{'description': 'Head Office', 'address1': '#4...",500000,Vancouver,49.275323,-123.120879,"{'type': 'Point', 'coordinates': [-123.1208785...",5,2,20,11
2,5fb0325b71eb61051a67a857,VideoClix,web,1999.0,"{'description': '', 'address1': '8th Floor', '...",9600000,Vancouver,49.283632,-123.104934,"{'type': 'Point', 'coordinates': [-123.104934,...",5,1,20,5


#### Ahora procedemos a ponderar. Para mi análisis he considerado que:
- Tener un aeropuerto internacional en un radio de 2 km tiene un peso de 0,3 porque al ganar tiempo minimiza costes para la empresa.
- Tener un colegio en un radio de 1 km tiene un peso de 0,2, porque el 30% de la plantilla tiene hijos.
- Tener un club nocturno a menos de 1 km tiene un peso de 0,3, porque teniendo en cuenta la edad de la plantilla podría interesar a todos.
- Tener un Starbucks a menos de 500 metros tiene un peso de 0,2 porque es importante que los ejecutivos no pierdan demasiado tiempo de trabajo yendo a por café.

#### Con esta información obtenemos la columna 'score' que ordenamos de forma descendente de modo que los primeros resultados son los de las oficinas que en base a los criterios explicados han obtenido una mayor puntuación.

In [21]:
filled_offices_df['score'] = (filled_offices_df["airports_2km"] * 0.3) + (filled_offices_df["primary_schools_1km"] *0.2) + (filled_offices_df["night_clubs_1km"] *0.3) + (filled_offices_df["starbucks_500m"] * (0.2))

In [22]:
filled_offices_df = filled_offices_df.sort_values("score", ascending=False)
filled_offices_df

,_id,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location,airports_2km,primary_schools_1km,night_clubs_1km,starbucks_500m,score
13,5fb0325c71eb61051a67b17d,Nexterra,cleantech,NaN,"{'description': 'Corporate Headquarters', 'add...",29000000,Vancouver,49.282159,-123.117650,"{'type': 'Point', 'coordinates': [-123.1176496...",5,3,20,20,12.1
4,5fb0325b71eb61051a67a870,BuildDirect,ecommerce,1999.0,"{'description': '', 'address1': '717 West Pend...",33000000,Vancouver,49.283935,-123.115930,"{'type': 'Point', 'coordinates': [-123.11593, ...",5,2,20,20,11.9
11,5fb0325c71eb61051a67b075,Ostara,cleantech,2005.0,"{'description': '', 'address1': '690 â€“ 1199 ...",45681891,Vancouver,49.288355,-123.123355,"{'type': 'Point', 'coordinates': [-123.1233545...",5,2,20,20,11.9
5,5fb0325b71eb61051a67a97d,Bootup Labs,consulting,2008.0,"{'description': '', 'address1': '163 West Hast...",262500,Vancouver,49.284608,-123.110394,"{'type': 'Point', 'coordinates': [-123.110394,...",5,1,20,16,10.9
14,5fb0325c71eb61051a67b18f,Tiny Speck,games_video,2009.0,"{'description': '', 'address1': '163 Hastings ...",17200000,Vancouver,49.282455,-123.109217,"{'type': 'Point', 'coordinates': [-123.109217,...",5,1,20,15,10.7
3,5fb0325b71eb61051a67a858,Strutta,advertising,2007.0,"{'description': '', 'address1': '128 West Hast...",1500000,Vancouver,49.282108,-123.107805,"{'type': 'Point', 'coordinates': [-123.1078045...",5,1,20,14,10.5
12,5fb0325c71eb61051a67b17b,FundRazr,social,2008.0,"{'description': 'Vancouver Office', 'address1'...",243750,Vancouver,49.284394,-123.109262,"{'type': 'Point', 'coordinates': [-123.1092622...",5,1,20,14,10.5
6,5fb0325b71eb61051a67aa31,Suite101,web,1996.0,"{'description': 'Vancouver HQ', 'address1': '2...",0,Vancouver,49.275332,-123.120893,"{'type': 'Point', 'coordinates': [-123.120893,...",5,2,20,12,10.3
1,5fb0325b71eb61051a67a68c,TeamPages,web,2006.0,"{'description': 'Head Office', 'address1': '#4...",500000,Vancouver,49.275323,-123.120879,"{'type': 'Point', 'coordinates': [-123.1208785...",5,2,20,11,10.1
0,5fb0325b71eb61051a67a560,NowPublic,web,2005.0,"{'description': '', 'address1': '576 Seymour S...",12000000,Vancouver,49.283376,-123.103986,"{'type': 'Point', 'coordinates': [-123.103986,...",5,2,20,5,8.9


In [23]:
top_3 = filled_offices_df.copy()

In [24]:
top_3 = top_3.head(3)

In [25]:
top_3

,_id,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location,airports_2km,primary_schools_1km,night_clubs_1km,starbucks_500m,score
13,5fb0325c71eb61051a67b17d,Nexterra,cleantech,NaN,"{'description': 'Corporate Headquarters', 'add...",29000000,Vancouver,49.282159,-123.117650,"{'type': 'Point', 'coordinates': [-123.1176496...",5,3,20,20,12.1
4,5fb0325b71eb61051a67a870,BuildDirect,ecommerce,1999.0,"{'description': '', 'address1': '717 West Pend...",33000000,Vancouver,49.283935,-123.115930,"{'type': 'Point', 'coordinates': [-123.11593, ...",5,2,20,20,11.9
11,5fb0325c71eb61051a67b075,Ostara,cleantech,2005.0,"{'description': '', 'address1': '690 â€“ 1199 ...",45681891,Vancouver,49.288355,-123.123355,"{'type': 'Point', 'coordinates': [-123.1233545...",5,2,20,20,11.9


In [32]:
top_3.to_csv(r'output/top_3_offices.csv')

### Aquí y para poder pintar el mapa vamos a reutilizar un par de funciones de los archivos de funciones para obtener las coordenadas del aeropuerto, el Starbucks, el nightclub y la escuela más cercanos a la oficina elegida.

In [95]:
def nearest_place(coordinates, places, place_type):
    min_distance = None
    nearest_place = None
    for place in places:
        place_coordinates = place["geometry"]["location"]
        distance_in_meters = fl.meters_between_coordinates(coordinates[1], coordinates[0], place_coordinates["lat"], place_coordinates["lng"])
        if not min_distance:
            min_distance = distance_in_meters
            nearest_place = place
        elif distance_in_meters < min_distance:
            min_distance = distance_in_meters
            nearest_place = place
    
    if nearest_place:
        return {"name": nearest_place["name"], 
                "location": nearest_place["geometry"]["location"], 
                "meters_to_office": min_distance,
                "type": place_type}
    else:
        return {}

In [96]:
def fill_nearest_places(coordinates):
    latlong = str(coordinates[::-1]).replace(' ', '')[1:-1]
    nearest_places = []
    airports_nearby = gl.get_google(latlong, radius=2000, typ='airport', apiKey=apiKey, keyw='international')
    nearest_places.append(nearest_place(coordinates, airports_nearby, 'airport'))
    
    primary_schools_nearby = gl.get_google(latlong, radius=1000, typ='primary_school', apiKey=apiKey)
    nearest_places.append(nearest_place(coordinates, primary_schools_nearby, 'primary_school'))
    
    
    night_clubs_nearby = gl.get_google(latlong, radius=1000, typ='night_club', apiKey=apiKey)
    nearest_places.append(nearest_place(coordinates, night_clubs_nearby, 'night_club'))

    starbucks_nearby = gl.get_google(latlong, radius=500, typ='cafe', apiKey=apiKey, keyw='starbucks')
    nearest_places.append(nearest_place(coordinates, starbucks_nearby, 'starbucks'))

    return nearest_places

In [97]:
nearest_places = fill_nearest_places([-123.117650, 49.282159])

In [98]:
nearest_places

[{'name': 'Vancouver airport',
  'location': {'lat': 49.2842096, 'lng': -123.1094381},
  'meters_to_office': 639.5119037045766,
  'type': 'airport'},
 {'name': 'Acting With NATHALIE THERRIAULT',
  'location': {'lat': 49.277642, 'lng': -123.1133467},
  'meters_to_office': 591.948925721164,
  'type': 'primary_school'},
 {'name': "Hy's Steakhouse & Cocktail Bar",
  'location': {'lat': 49.2845532, 'lng': -123.1191216},
  'meters_to_office': 286.9911945088801,
  'type': 'night_club'},
 {'name': 'Starbucks',
  'location': {'lat': 49.28207450000001, 'lng': -123.1184392},
  'meters_to_office': 58.18427853798812,
  'type': 'starbucks'}]